### ReLU DNN Example

#### Load and prepared data

In [2]:
import numpy as np
import pandas as pd
from modeva import DataSet
# Loading data into pandas dataframe
df = pd.read_csv("credit_default.csv")
df.columns = ["employment",	"income", "dti", "score", "amount", 
              "tenure",	"emp_length", "delinquencies",
              "savings", "utilization", "default"]

In [3]:
# Create dataset object holder
ds = DataSet()
ds.load_dataframe(data = df) 

✓ Auth code found in local storage.
Authenticating Modeva...
✓ License is active and valid.
✓ Authenticated successfully!


In [4]:
## Preprocess the data
ds.encode_categorical(method="ordinal")
ds.scale_numerical(features=tuple(ds.feature_names_numerical), method="standardize") # standardized numerical features
ds.set_target('default')
ds.preprocess()
## Split data into training and testing sets randomly
ds.set_random_split() 

#### ReLU DNN

In [5]:
from modeva.models import MoReLUDNNClassifier
model_relunet = MoReLUDNNClassifier(name = "ReLU_Net", hidden_layer_sizes=(20, 20), l1_reg=0.0002, learning_rate=0.001)
# train model with input: ds.train_x and target: ds.train_y
model_relunet.fit(ds.train_x, ds.train_y.ravel())

MoReLUDNNClassifier(device='cpu', hidden_layer_sizes=(20, 20), l1_reg=0.0002,
                    name='ReLU_Net')

In [6]:
# Create a testsuite that bundles dataset and model
from modeva import TestSuite
ts = TestSuite(ds, model_relunet) # store bundle of dataset and model in fs
# View model performance metrics
result = ts.diagnose_accuracy_table()
# display the output
result.table

,AUC,ACC,F1,LogLoss,Brier
train,0.923935,0.879125,0.697907,0.277712,0.085902
test,0.925235,0.880000,0.700000,0.272522,0.084688
GAP,0.001300,0.000875,0.002093,-0.005190,-0.001214


#### Local Linear Model Summary

In [7]:
# Summary of local linear models
result = ts.interpret_llm_summary()
# display the output
result.table

,Count,Response Mean,Response Std,Local AUC,Global AUC
0,55.0,0.1273,0.3363,0.9196,0.2344
1,16.0,0.0000,0.0000,NaN,0.7755
2,29.0,0.1379,0.3509,0.8800,0.2320
3,29.0,0.1379,0.3509,0.9300,0.2351
4,19.0,0.3684,0.4956,0.8810,0.2329
...,...,...,...,...,...
2707,1.0,1.0000,NaN,NaN,0.7718
2708,1.0,1.0000,NaN,NaN,0.7640
2709,1.0,0.0000,NaN,NaN,0.7762
2710,1.0,1.0000,NaN,NaN,0.2436


##### Parallel Coordinate Plot

In [8]:
# Parallel coordinate of LLM coeficients
result = ts.interpret_llm_pc()
# Plot the result
result.plot()

##### Global feature importance

In [9]:
results = ts.interpret_fi()
results.plot()

##### Profile plot for localalized effects

In [10]:
results = ts.interpret_llm_profile(feature="score", dataset="train")
results.plot()

##### ReLU-Net with higher regularization

In [11]:
model_relunet_reg = MoReLUDNNClassifier(name = "ReLU_Net_Regularized", hidden_layer_sizes=(20, 20), l1_reg=0.002, learning_rate=0.001)
# train model with input: ds.train_x and target: ds.train_y
model_relunet_reg.fit(ds.train_x, ds.train_y.ravel())

MoReLUDNNClassifier(device='cpu', hidden_layer_sizes=(20, 20), l1_reg=0.002,
                    name='ReLU_Net_Regularized')

In [12]:
tsc = TestSuite(ds, models=[model_relunet, model_relunet_reg])
results = tsc.compare_accuracy_table()
results.table

ReLU_Net                                 ReLU_Net_Regularized          \
           AUC     ACC      F1 LogLoss   Brier                  AUC     ACC   
train   0.9239  0.8791  0.6979  0.2777  0.0859               0.9201  0.8764   
test    0.9252  0.8800  0.7000  0.2725  0.0847               0.9237  0.8810   
GAP     0.0013  0.0009  0.0021 -0.0052 -0.0012               0.0036  0.0046   

                               
           F1 LogLoss   Brier  
train  0.6853  0.2848  0.0879  
test   0.6949  0.2757  0.0854  
GAP    0.0095 -0.0091 -0.0025

In [13]:
ts2 = TestSuite(ds, model_relunet_reg) 
# Parallel coordinate of LLM coeficients
result = ts2.interpret_llm_pc()
# Plot the result
result.plot()

#### LLM Summary Table

In [14]:
# Summary of local linear models
result = ts2.interpret_llm_summary()
# display the output
result.table

,Count,Response Mean,Response Std,Local AUC,Global AUC
0,1547.0,0.1584,0.3652,0.8966,0.7778
1,124.0,0.0887,0.2855,0.9533,0.2306
2,149.0,0.4765,0.5011,0.8442,0.2305
3,76.0,0.0000,0.0000,NaN,0.2306
4,33.0,0.9091,0.2919,0.6778,0.6300
5,24.0,0.8750,0.3378,0.7937,0.7768
6,5.0,1.0000,0.0000,NaN,0.2305
7,10.0,0.9000,0.3162,0.4444,0.7768
8,4.0,1.0000,0.0000,NaN,0.7325
9,10.0,1.0000,0.0000,NaN,0.7770
